In [1]:
import numpy as np
from scipy import stats
np.random.seed(42)

# Simulated checkout times (seconds) for 3 designs, n≈20 each
A = np.random.normal(loc=72, scale=8, size=20)  # Design A
B = np.random.normal(loc=68, scale=7, size=20)  # Design B
C = np.random.normal(loc=62, scale=7, size=20)  # Design C

# Assumption checks (normality per group; homogeneity of variance)
shapiro_A = stats.shapiro(A)
shapiro_B = stats.shapiro(B)
shapiro_C = stats.shapiro(C)
levene_AB = stats.levene(A, B, C)

# One-way ANOVA
F, p = stats.f_oneway(A, B, C)

# Effect size (eta-squared): SS_between / SS_total
grand = np.concatenate([A, B, C]).mean()
ss_between = len(A)*(A.mean()-grand)**2 + len(B)*(B.mean()-grand)**2 + len(C)*(C.mean()-grand)**2
ss_total = ((np.concatenate([A,B,C]) - grand)**2).sum()
eta_sq = ss_between / ss_total

print("Shapiro p-values:", shapiro_A.pvalue, shapiro_B.pvalue, shapiro_C.pvalue)
print("Levene p-value:", levene_AB.pvalue)
print("ANOVA: F =", round(F,3), "p =", p)
print("Eta-squared =", round(eta_sq,3))

Shapiro p-values: 0.8478800044350963 0.7675616535134637 0.32706435690809055
Levene p-value: 0.6306937778256522
ANOVA: F = 8.455 p = 0.0006087712838764671
Eta-squared = 0.229


In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.DataFrame({
    "time": np.concatenate([A, B, C]),
    "design": (["A"]*len(A))+ (["B"]*len(B)) + (["C"]*len(C))
})
tukey = pairwise_tukeyhsd(endog=df["time"], groups=df["design"], alpha=0.05)
print(tukey)


 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     A      B  -4.4914 0.0999  -9.6511  0.6682  False
     A      C  -8.8164 0.0004 -13.9761 -3.6568   True
     B      C   -4.325 0.1173  -9.4847  0.8347  False
-----------------------------------------------------
